<a href="https://colab.research.google.com/github/Ashutosh-Bahet/Commidty/blob/main/BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')

  print('there are %d GPU(s) available.' % torch.cuda.device_count())

  print('we will use the GPU: ', torch.cuda.get_device_name(0))

else:
  print("No GPU available, using the CPU instead")
  device = torch.device("cpu")

In [ ]:
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')

  print('there are %d GPU(s) available.' % torch.cuda.device_count())

  print('we will use the GPU: ', torch.cuda.get_device_name(0))

else:
  print("No GPU available, using the CPU instead")
  device = torch.device("cpu")
import pandas as pd

df = pd.read_csv('/content/gold-dataset-sinha-khandait.csv', usecols=["Price Sentiment", "News"])
df.rename(columns = {'News':'news', 'Price Sentiment':'sentiment'}, inplace = True)
df.info()
df = df[(df.sentiment != "none")]
df = df.replace({'sentiment' : {'negative':0, 'neutral':1, 'positive':2}})
df.head()
import seaborn as sns
import matplotlib.pyplot as plt

sentiment = df['sentiment'].value_counts()
plt.figure(figsize=(12,4))
sns.barplot(x=sentiment.index, y=sentiment.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('sentiment', fontsize=12)
plt.xticks(rotation=90)
plt.show();
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from bs4 import BeautifulSoup
import contractions
import tqdm

def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Fix contractions (English only)
    text = contractions.fix(text)

    # Strip whitespace
    text = text.strip()

    return text


def pre_process_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
        cleaned = clean_text(doc)
        norm_docs.append(cleaned)
    return norm_docs
import nltk
nltk.download('punkt_tab')
df.news = pre_process_corpus(df.news)
sentences = df.news.values
labels = df.sentiment.values
from transformers import BertTokenizer

print("load BERT Tokenizer")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_cased = True)
encoding = tokenizer(
    sentences.tolist(),
    max_length=256,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
print('Original:', sentences[0])
print('Token IDS:', input_ids[0])
print('Max Sentence length', max([len(sen) for sen in input_ids]))
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']
from sklearn.model_selection import train_test_split

# 1. Train / Test split
train_input, test_input, train_mask, test_mask, train_labels, test_labels = train_test_split(
    input_ids, attention_mask, labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

# 2. Train / Validation split
train_input, val_input, train_mask, val_mask, train_labels, val_labels = train_test_split(
    train_input, train_mask, train_labels,
    test_size=0.1,
    random_state=42,
    stratify=train_labels
)
import numpy as np

print('---Train---')
print('input: ', train_input.shape)
print('label: ', train_labels.shape)
print('mask: ', train_mask.shape)

print('---Validation---')
print('input: ', val_input.shape)
print('label: ', val_labels.shape)
print('mask: ', val_mask.shape)

print('---Test---')
print('input: ', test_input.shape)
print('label: ', test_labels.shape)
print('mask: ', test_mask.shape)
import torch

train_input = torch.tensor(train_input)
train_labels = torch.tensor(train_labels)
train_mask   = torch.tensor(train_mask)

val_input = torch.tensor(val_input)
val_labels = torch.tensor(val_labels)
val_mask   = torch.tensor(val_mask)

test_input = torch.tensor(test_input)
test_labels = torch.tensor(test_labels)
test_mask   = torch.tensor(test_mask)
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 10
train_data = TensorDataset(train_input, train_mask, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
val_data = TensorDataset(val_input, val_mask, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)
test_data = TensorDataset(test_input, test_mask, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
from transformers import BertForSequenceClassification,get_linear_schedule_with_warmup
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))
import time
import datetime
model.to(device)
gradient_accumulation_steps = 4

for epoch_i in range(epochs):
    print(f"Epoch {epoch_i + 1}/{epochs}")
    print("Training...")

    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        outputs = model(
            b_input_ids,
            attention_mask=b_input_mask,
            labels=b_labels
        )

        loss = outputs.loss
        loss = loss / gradient_accumulation_steps
        total_loss += loss.item()

        loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss:.4f}")
    print(f"Training took: {format_time(time.time() - t0)}")

    # ------------------------------
    # Validation
    # ------------------------------
    print("Running Validation...")
    t0 = time.time()
    model.eval()

    eval_accuracy = 0
    eval_loss = 0
    nb_eval_steps = 0

    for batch in val_dataloader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(
                b_input_ids,
                attention_mask=b_input_mask,
                labels=b_labels
            )

        loss = outputs.loss
        logits = outputs.logits

        eval_loss += loss.item()

        preds = logits.detach().cpu().numpy()
        label_ids = b_labels.to("cpu").numpy()

        eval_accuracy += flat_accuracy(preds, label_ids)
        nb_eval_steps += 1

    print(f"Validation Loss: {eval_loss / nb_eval_steps:.4f}")
    print(f"Validation Accuracy: {eval_accuracy / nb_eval_steps:.4f}")
    print(f"Validation took: {format_time(time.time() - t0)}\n")

print("Training complete!")
model.eval()

all_preds = []
all_labels = []

for batch in test_dataloader:

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():
        outputs = model(
            b_input_ids,
            attention_mask=b_input_mask
        )

    logits = outputs.logits
    preds = torch.argmax(logits, dim=1)

    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(b_labels.cpu().numpy())
    from sklearn.metrics import matthews_corrcoef

mcc = matthews_corrcoef(all_labels, all_preds)
print("MCC:", mcc)



In [ ]:
!pip install contractions

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

df = pd.read_csv('/content/gold-dataset-sinha-khandait.csv', usecols=["Price Sentiment", "News"])
df.rename(columns = {'News':'news', 'Price Sentiment':'sentiment'}, inplace = True)
df.info()

In [ ]:
df = df[(df.sentiment != "none")]
df = df.replace({'sentiment' : {'negative':0, 'neutral':1, 'positive':2}})
df.head()

In [ ]:
df.info()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sentiment = df['sentiment'].value_counts()
plt.figure(figsize=(12,4))
sns.barplot(x=sentiment.index, y=sentiment.values, alpha=0.8)
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('sentiment', fontsize=12)
plt.xticks(rotation=90)
plt.show();

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')


In [ ]:
from bs4 import BeautifulSoup
import contractions
import tqdm

def clean_text(text):
    # Remove HTML tags
    text = BeautifulSoup(text, "html.parser").get_text()

    # Fix contractions (English only)
    text = contractions.fix(text)

    # Strip whitespace
    text = text.strip()

    return text


def pre_process_corpus(docs):
    norm_docs = []
    for doc in tqdm.tqdm(docs):
        cleaned = clean_text(doc)
        norm_docs.append(cleaned)
    return norm_docs


In [ ]:
import nltk
nltk.download('punkt_tab')

In [ ]:
df.news = pre_process_corpus(df.news)

In [ ]:
df.head()

In [ ]:
sentences = df.news.values
labels = df.sentiment.values

In [ ]:
from transformers import BertTokenizer

print("load BERT Tokenizer")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_cased = True)

In [ ]:
encoding = tokenizer(
    sentences.tolist(),
    max_length=256,
    padding='max_length',
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)

input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [ ]:
print('Original:', sentences[0])
print('Token IDS:', input_ids[0])

In [ ]:
print('Max Sentence length', max([len(sen) for sen in input_ids]))

In [ ]:
input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

In [ ]:
from sklearn.model_selection import train_test_split

# 1. Train / Test split
train_input, test_input, train_mask, test_mask, train_labels, test_labels = train_test_split(
    input_ids, attention_mask, labels,
    test_size=0.2,
    random_state=42,
    stratify=labels
)

# 2. Train / Validation split
train_input, val_input, train_mask, val_mask, train_labels, val_labels = train_test_split(
    train_input, train_mask, train_labels,
    test_size=0.1,
    random_state=42,
    stratify=train_labels
)


In [ ]:
import numpy as np

print('---Train---')
print('input: ', train_input.shape)
print('label: ', train_labels.shape)
print('mask: ', train_mask.shape)

print('---Validation---')
print('input: ', val_input.shape)
print('label: ', val_labels.shape)
print('mask: ', val_mask.shape)

print('---Test---')
print('input: ', test_input.shape)
print('label: ', test_labels.shape)
print('mask: ', test_mask.shape)


In [ ]:
import torch

train_input = torch.tensor(train_input)
train_labels = torch.tensor(train_labels)
train_mask   = torch.tensor(train_mask)

val_input = torch.tensor(val_input)
val_labels = torch.tensor(val_labels)
val_mask   = torch.tensor(val_mask)

test_input = torch.tensor(test_input)
test_labels = torch.tensor(test_labels)
test_mask   = torch.tensor(test_mask)


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 10
train_data = TensorDataset(train_input, train_mask, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
val_data = TensorDataset(val_input, val_mask, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)
test_data = TensorDataset(test_input, test_mask, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)


In [ ]:
from transformers import BertForSequenceClassification,get_linear_schedule_with_warmup
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)


In [ ]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 3
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import time
import datetime
model.to(device)
gradient_accumulation_steps = 4

for epoch_i in range(epochs):
    print(f"Epoch {epoch_i + 1}/{epochs}")
    print("Training...")

    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        outputs = model(
            b_input_ids,
            attention_mask=b_input_mask,
            labels=b_labels
        )

        loss = outputs.loss
        loss = loss / gradient_accumulation_steps
        total_loss += loss.item()

        loss.backward()

        if (step + 1) % gradient_accumulation_steps == 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            model.zero_grad()

    avg_train_loss = total_loss / len(train_dataloader)
    print(f"Average training loss: {avg_train_loss:.4f}")
    print(f"Training took: {format_time(time.time() - t0)}")

    # ------------------------------
    # Validation
    # ------------------------------
    print("Running Validation...")
    t0 = time.time()
    model.eval()

    eval_accuracy = 0
    eval_loss = 0
    nb_eval_steps = 0

    for batch in val_dataloader:

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            outputs = model(
                b_input_ids,
                attention_mask=b_input_mask,
                labels=b_labels
            )

        loss = outputs.loss
        logits = outputs.logits

        eval_loss += loss.item()

        preds = logits.detach().cpu().numpy()
        label_ids = b_labels.to("cpu").numpy()

        eval_accuracy += flat_accuracy(preds, label_ids)
        nb_eval_steps += 1

    print(f"Validation Loss: {eval_loss / nb_eval_steps:.4f}")
    print(f"Validation Accuracy: {eval_accuracy / nb_eval_steps:.4f}")
    print(f"Validation took: {format_time(time.time() - t0)}\n")

print("Training complete!")


In [ ]:
model.eval()

all_preds = []
all_labels = []

for batch in test_dataloader:

    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    with torch.no_grad():
        outputs = model(
            b_input_ids,
            attention_mask=b_input_mask
        )

    logits = outputs.logits
    preds = torch.argmax(logits, dim=1)

    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(b_labels.cpu().numpy())


In [ ]:
from sklearn.metrics import matthews_corrcoef

mcc = matthews_corrcoef(all_labels, all_preds)
print("MCC:", mcc)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(all_labels, all_preds)
print("Accuracy:", accuracy)
